# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [9]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [10]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [11]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#

'''
提示：可參考以下的Stack Overflow: 
   
https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
'''
    
count = 0
while True:
    time.sleep(2)
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, "html5lib")
    items = soup.find('div',{'class':'part_pictxt_3 lazyload'}).find_all('div',{'class':'piece clearfix'})
    if len(items) != count:
        count = len(items)
        print('網頁更新中...')
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight),10000;")
    else:
        print('到達頁面底端')
        break

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [12]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [13]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


怡／外電報導
Google去年在行動版的搜尋引擎做了些調整，就是將網址、圖示顯示在搜尋結果前端，幫搜尋者了解這訊息是哪來的。現在，Google於官方推特發文預告，關鍵字搜尋結果的全新排版介面，即將在桌機網頁版上線。
此次搜尋結果頁面的介面改版調整，包括搜尋結果列表的顯示內容資訊，把原本使用綠色細體字顯示的 URL 連結網址，改為「黑色」並移至上方。
另外，在每個網址的最前面，新增網站LOGO圖標，讓民眾可以判斷這訊息來源有沒有問題，內容是真是假。
還有，原本是使用綠色加框顯示為「廣告」字體的標籤，新版把「廣告」兩個字，改為無框粗體黑色的顯示方式。
▼Google新改版的搜尋頁面。（圖／翻攝自Google）
----------------------------------------------------------------------

[66] Apple Card只提供PDF檔對帳單　他自建工具供網友免費轉檔

▲日前有開發人員為此推出了一項新工具，能將Apple Card輸出的PDF對帳單轉檔成CSV格式。（圖／翻攝自Apple）
記者王曉敏／綜合報導
根據蘋果信用卡服務Apple Card的設計，是將用戶消費歷史記載在一個豐富多彩的互動式介面電子錢包App中，但若想要在Excel等試算表上分析自己的交易就麻煩了，因為其無法輸出CSV檔案。不過日前有開發人員為此推出了一項新工具，能輕鬆解決問題。
「csv.wtf」實際上是一款Web應用程式，用戶僅需從iPhone的「錢包」App下載對帳單，並將該PDF檔案上傳至「csv.wtf」網站上，即可生成該對帳單的CSV檔案。其不僅能在電腦上操作，也能在iPhone上使用。
施密特皆受CNBC採訪時表示，他很早前就申請了Apple Card，「讓我驚訝的是它竟然沒有提供CSV檔，」他表示， 「在我看來，蘋果、高盛、萬事達不會發布這種無法輸出交易的產品。這真的令人驚訝。」
CNBC指出，這並非Apple Card第一個缺乏某些標準功能的例子。根據MarketWatch的報導，去年Apple Card剛推出的前幾個月，蘋果和高盛都未向主要徵信季購報告消費者在Apple Card上的支付訊息。直到第四季稍晚，Apple Card使用者的付款紀錄才開始反映在消費者的信用報告上。
目前「csv.wtf」網站可供消費者免